<a href="https://colab.research.google.com/github/foramirez50/DeepLearning/blob/main/Rostami_multiscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import *
from keras.layers import *

In [5]:
def conv2d_block(input_tensor, kernel_size, n_filters, batchnorm = True):
  
  xA = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",padding="same")(input_tensor)
  xA = BatchNormalization()(xA)
  xA = Activation("relu")(xA)

  xB = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",padding="same")(input_tensor)
  xB = BatchNormalization()(xB)

  x_concat = Concatenate([xA, xB])
  x_concat = Activation("relu")(x_concat)

  return x_concat

In [10]:
def get_multiscale(nClasses, input_height=256, input_width=256, n_filters = 16, dropout = 0.1, batchnorm = True):

          input_height=265
          input_width=265
          img_input = Input(shape=(input_height,input_width, 4))


          # contracting path

          # Encoder layer 1 
          layer1 = Concatenate ([
              conv2d_block(img_input, 3, n_filters*1,batchnorm = True),
              conv2d_block(img_input, 5, n_filters*1,batchnorm = True),
              conv2d_block(img_input, 7, n_filters*1,batchnorm = True)
              ]
          )
          layer1 = MaxPooling2D((2, 2)) (layer1)

          # Encoder layer 2

          layer2 = Concatenate ([
              conv2d_block(layer1, 3, n_filters*2,batchnorm = True),
              conv2d_block(layer1, 5, n_filters*2,batchnorm = True),
              conv2d_block(layer1, 7, n_filters*2,batchnorm = True)
              ]
          )
          layer2 = MaxPooling2D((2, 2)) (layer2)

          # Encoder layer 3

          layer3 = Concatenate ([
              conv2d_block(layer2, 3, n_filters*4,batchnorm = True),
              conv2d_block(layer2, 5, n_filters*4,batchnorm = True),
              conv2d_block(layer2, 7, n_filters*4,batchnorm = True)
              ]
          )
          layer3= MaxPooling2D((2, 2)) (layer3)

          # Encoder layer 4

          layer4 = Concatenate ([
              conv2d_block(layer3, 3, n_filters*8,batchnorm = True),
              conv2d_block(layer3, 5, n_filters*8,batchnorm = True),
              conv2d_block(layer3, 7, n_filters*8,batchnorm = True)
              ]
          )
          layer4= MaxPooling2D((2, 2)) (layer4)

          # Encoder layer 5

          layer5 = Concatenate ([
              conv2d_block(layer4, 3, n_filters*16,batchnorm = True),
              conv2d_block(layer4, 5, n_filters*16,batchnorm = True),
              conv2d_block(layer4, 7, n_filters*16,batchnorm = True)
              ]
          )
          layer5= MaxPooling2D((2, 2)) (layer5)


          # expansive path

          # Decoder layer 1

          d_layer1 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same') (layer5)
          d_layer1 = BatchNormalization()(d_layer1)
          d_layer1 = Activation("relu")(d_layer1)
          d_layer1 = Concatenate([d_layer1, layer4])

          d_layer1 = Concatenate ([
              conv2d_block(d_layer1, 3, n_filters*8,batchnorm = True) ,
              conv2d_block(d_layer1, 5, n_filters*8,batchnorm = True) ,
              conv2d_block(d_layer1, 7, n_filters*8,batchnorm = True)
              ]
          )

          # Decoder layer 2

          d_layer2 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same') (d_layer1)
          d_layer2 = BatchNormalization()(d_layer2)
          d_layer2 = Activation("relu")(d_layer2)
          d_layer2 = Concatenate([d_layer2, layer3])

          d_layer2 = Concatenate ([
              conv2d_block(d_layer2, 3, n_filters*4,batchnorm = True) ,
              conv2d_block(d_layer2, 5, n_filters*4,batchnorm = True) ,
              conv2d_block(d_layer2, 7, n_filters*4,batchnorm = True)
              ]
          )

          # Decoder layer 3

          d_layer3 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same') (d_layer2)
          d_layer3 = BatchNormalization()(d_layer3)
          d_layer3 = Activation("relu")(d_layer3)
          d_layer3 = Concatenate([d_layer3, layer2])

          d_layer3 = Concatenate ([
              conv2d_block(d_layer3, 3, n_filters*2,batchnorm = True) ,
              conv2d_block(d_layer3, 5, n_filters*2,batchnorm = True) ,
              conv2d_block(d_layer3, 7, n_filters*2,batchnorm = True)
              ]
          )

          # Decoder layer 4

          d_layer4 = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same') (d_layer2)
          d_layer4 = BatchNormalization()(d_layer4)
          d_layer4 = Activation("relu")(d_layer4)
          d_layer4 = Concatenate([d_layer4, layer1])

          d_layer4 = Concatenate ([
              conv2d_block(d_layer4, 3, n_filters,batchnorm = True) ,
              conv2d_block(d_layer4, 5, n_filters,batchnorm = True) ,
              conv2d_block(d_layer4, 7, n_filters,batchnorm = True)
              ]
          )

          d_layer4 = Dropout(dropout)(d_layer4)      


          # output layer
          outputs = Conv2D(1, (1, 1), activation='softmax') (d_layer4)
          return outputs   